In [350]:
import random
import math

In [351]:
def read_input_from_file(file_path):
  with open(file_path, 'r') as file:
    # Ler a primeira linha que contém N
    N = int(file.readline().strip())  # Quantidade de itens
    
    # Ler a segunda linha que contém C
    C = int(file.readline().strip())  # Capacidade de cada caixa
    
    # Ler os pesos dos itens, um por linha
    item_weights = []
    for _ in range(N):
      weight = int(file.readline().strip())
      item_weights.append(weight)
  
  return N, C, item_weights

In [352]:
def write_output_to_file(file_path, best_fitness):
  with open(file_path, 'w') as file:
    file.write(f'Numero de bins usados (fitness): {best_fitness}\n')

In [353]:
input_file = 'input.in'
output_file = 'output.out'

n, C, bin_sizes = read_input_from_file(input_file)

In [354]:
def check(now):
  need = 1
  total_sum = 0
  for i in range(n):
    if total_sum + bin_sizes[now[i]] <= C:
      total_sum += bin_sizes[now[i]]
    else:
      total_sum = bin_sizes[now[i]]
      need += 1
  return need

In [355]:
cooling_rate = 1 - math.exp(-n / 100) # quanto maior o N mais proximo de 1.0
taxa = 100 * n                        # taxa de embaralhamentos, quanto maior o numero de caixas menor sera o numero de shuffles para testar
out = n // 9                          # numero de caixas alem da testada atualmente(delta) que é aceitavel no momento, ela muda com base no cooling rate
global_solution = n                   # a melhor solução inicial será um item inserido em cada caixa(inicializa com o pior possivel)

def try_packs(packs):

  global out
  global global_solution
  global cooling_rate
  
  out = out * cooling_rate            # diferentemente do SA neste caso o cooling rate neste caso nao varia com o tempo
                                      # contudo o cooling rate diminuiu o valor de "out" a cada iteração diminuindo o alcance de aceitação a cada processo de iteração
  tests = taxa // packs               # quanto maior o numero de packs menor sera a quantidade de shuffles a serem testados(teste de tecnica gulosa)

  ok = False

  for _ in range(tests):
    permutation = list(range(n))
    random.shuffle(permutation)

    need = check(permutation)
      
    if need <= packs + out:     # se o numero de caixas necessarias atualmente é <= ao numero de caixas aceitavel + delta(dependendo da temperatura atual do sistema) 
      ok = True                 # quanto maior a temperatura maior sera o "out" levando a aceitar uma solução mesmo que seja "ruim" no momento
    
    global_solution = min(global_solution, need)

  return ok

In [356]:
def main():

  global cooling_rate
  global global_solution

  l, r = 1, n
  while r - l > 1:
    bins = (l + r) // 2
    if try_packs(bins) or global_solution < bins:    # se alguma configuração esteve dentro do limite  OU
      r = bins                                       # se alguma solução encontrada ja foi menor que a quantidade de caixas atual, eu diminuo a quantidade de caixas a ser testada
      global_solution = min(global_solution, bins)
    else:
      l = bins + 1

  write_output_to_file(output_file, global_solution)

if __name__ == "__main__":
  main()